In [1]:
import cv2
import numpy as np
import logging
import time 
import user_dataclasses
from user_input import UserInput
from video_data import VideoData
from dataclasses import dataclass, field
from moviepy.editor import VideoFileClip, concatenate_videoclips
from tqdm import tqdm

In [2]:
vd = VideoData("content/spikeball5.mov", "spikeball-nn.h5")

2021-11-30 10:50:19,303 - VideoData - INFO - 	VideoData created
2021-11-30 10:50:19.375047: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
bal_in_frame = vd.get_ball_bounce_frames()

2021-11-30 10:50:24,406 - UserInput - INFO - 	UserInput created
2021-11-30 10:50:24,407 - UserInput - INFO - 	Please select the upper left and bottom right corner of the spikeball net bounding box
2021-11-30 10:50:37,771 - UserInput - INFO - 	Points selected: [(848, 520), (1020, 633)]
2021-11-30 10:50:37,772 - UserInput - INFO - 	The selected rectangle is now shown on the display
2021-11-30 10:50:37,777 - UserInput - INFO - 	If you would like to redo the points then please press r, otherwise click any other key
2021-11-30 10:50:47,797 - VideoData - INFO - 	Bounding box: 823, 495, 1045, 658
2021-11-30 10:50:47,800 - VideoData - INFO - 	Created foreground mask
100%|█████████▉| 64920/64950 [08:16<00:00, 130.83it/s]


In [4]:
balls_in_frame = bal_in_frame
balls_in_frame

[3,
 384,
 385,
 386,
 387,
 388,
 1126,
 1128,
 1129,
 1130,
 1131,
 1260,
 1261,
 1262,
 1263,
 1267,
 1268,
 1269,
 1270,
 1271,
 1272,
 1273,
 1274,
 1306,
 1309,
 1310,
 1311,
 1539,
 1543,
 1546,
 1549,
 2329,
 2330,
 2331,
 2334,
 2335,
 2336,
 2337,
 2338,
 2554,
 3032,
 3371,
 3372,
 3373,
 3374,
 3375,
 3376,
 3377,
 3378,
 3985,
 3987,
 3988,
 3989,
 4066,
 4067,
 4070,
 4071,
 4075,
 4076,
 4077,
 4078,
 4079,
 4081,
 4082,
 4083,
 4084,
 4085,
 4086,
 4363,
 4364,
 4367,
 4368,
 4369,
 4440,
 4441,
 4442,
 4443,
 4444,
 4445,
 4446,
 4455,
 4456,
 4457,
 4459,
 4460,
 4461,
 5227,
 5228,
 5229,
 5230,
 5231,
 5232,
 5233,
 5234,
 5235,
 5236,
 5237,
 5238,
 5497,
 5498,
 5499,
 5503,
 5647,
 5653,
 5774,
 5777,
 6456,
 6461,
 6462,
 6463,
 6464,
 6644,
 6645,
 6646,
 7529,
 7530,
 7531,
 7534,
 7535,
 7536,
 8067,
 8068,
 8069,
 8070,
 8073,
 8074,
 8075,
 8076,
 8206,
 8207,
 10507,
 12717,
 13129,
 13130,
 13131,
 13132,
 13133,
 13134,
 13137,
 13138,
 13139,
 13140,
 1

In [5]:
@dataclass
class Interval:
    start: int
    end: int
    frames: list[int] = field(default_factory=list)

In [6]:
time_before_contact = 1.0
time_after_contact = 6
frames_before_contact = time_before_contact * 60
frames_after_contact = time_after_contact * 60

In [7]:
intervals: list[Interval] = []

inside = False
prev = None
for frame_number in balls_in_frame:
    if prev is None:
        prev = frame_number
        intervals.append(Interval(start=max(frame_number - frames_before_contact, 0), end=None))
        intervals[-1].frames.append(frame_number)
        continue
    if frame_number - prev > frames_after_contact:
        intervals[-1].end = prev + frames_after_contact
        inside = False
        intervals.append(Interval(start=frame_number - frames_before_contact, end=None))
        intervals[-1].frames.append(frame_number)
    else:
        intervals[-1].frames.append(frame_number)
    prev = frame_number

In [8]:
intervals
#len(intervals)

[Interval(start=0, end=363, frames=[3]),
 Interval(start=324.0, end=748, frames=[384, 385, 386, 387, 388]),
 Interval(start=1066.0, end=1909, frames=[1126, 1128, 1129, 1130, 1131, 1260, 1261, 1262, 1263, 1267, 1268, 1269, 1270, 1271, 1272, 1273, 1274, 1306, 1309, 1310, 1311, 1539, 1543, 1546, 1549]),
 Interval(start=2269.0, end=2914, frames=[2329, 2330, 2331, 2334, 2335, 2336, 2337, 2338, 2554]),
 Interval(start=2972.0, end=3738, frames=[3032, 3371, 3372, 3373, 3374, 3375, 3376, 3377, 3378]),
 Interval(start=3925.0, end=4821, frames=[3985, 3987, 3988, 3989, 4066, 4067, 4070, 4071, 4075, 4076, 4077, 4078, 4079, 4081, 4082, 4083, 4084, 4085, 4086, 4363, 4364, 4367, 4368, 4369, 4440, 4441, 4442, 4443, 4444, 4445, 4446, 4455, 4456, 4457, 4459, 4460, 4461]),
 Interval(start=5167.0, end=6137, frames=[5227, 5228, 5229, 5230, 5231, 5232, 5233, 5234, 5235, 5236, 5237, 5238, 5497, 5498, 5499, 5503, 5647, 5653, 5774, 5777]),
 Interval(start=6396.0, end=7006, frames=[6456, 6461, 6462, 6463, 6464, 

In [9]:
clips = []
video = VideoFileClip("content/spikeball5.mov")
for interval in tqdm(intervals):
    if interval.end is not None:
        clips.append(video.subclip(interval.start//60, interval.end//60))
#clips = [c for c in clips if c is not None]

final_clip = concatenate_videoclips(clips)

100%|██████████| 60/60 [00:52<00:00,  1.14it/s]


In [10]:
final_clip.write_videofile("content/spikeball5_intervals2.mp4", threads=8, audio=False, fps=60)
final_clip.close()

Moviepy - Building video content/spikeball5_intervals2.mp4.
Moviepy - Writing video content/spikeball5_intervals2.mp4



Moviepy - Done !
Moviepy - video ready content/spikeball5_intervals2.mp4
